In [ ]:
import os
import sys
import tiktoken
import concurrent.futures
import asyncio
import time
import logging
import pandas as pd
import json
import re
from typing import Any, List, Dict, Optional
from openai import RateLimitError

# from openai.error import RateLimitError
import backoff

In [ ]:
os.getcwd()

In [ ]:
import sys
sys.path.append('/libraries/')

In [ ]:
import libraries.llm_functions as lf

In [ ]:
# Import modules from libraries
try:
    from libraries import llm_functions as lf
    from libraries import neo4j_lib as nl
    from libraries import claude_prompts as cp
except ImportError as e:
    print(f"Error importing modules: {e}")
    sys.exit(1)

from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core import VectorStoreIndex, Document

from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from tqdm import tqdm

# Set up logging

# Model and memory configurations
llm = Ollama(
    model="llama3.1:latest", temperature=0, max_tokens=64768, request_timeout=120.0
)
llm = OpenAI(temperature=0, model="gpt-4o-mini", request_timeout=120.0)
MEMORY = ChatMemoryBuffer.from_defaults(token_limit=8192)

In [ ]:
def create_chat_engine(advert: str) -> Optional[Any]:
    documents = [Document(text=advert)]
    index = VectorStoreIndex.from_documents(documents)
    return index.as_chat_engine(
        chat_mode="context",
        llm=llm,
        memory=MEMORY,
        system_prompt=(
            "As a career forensic analyst, you have deep insight into crime "
            "and criminal activity, especially human trafficking. Investigate "
            "the online recruitment advert and extract pertinent details."
        ),
    )


In [ ]:
IDn = 573388
advert = nl.get_neo4j_advert(IDn)

In [ ]:
prompt_name='assure_prompt'
chat_engine = create_chat_engine(advert)
analysis = lf.analyse_advert(chat_engine, prompt_name)